In [3]:
import wikipedia
import pandas as pd
from time import sleep
import time
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.stem import PorterStemmer
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
import gensim

wikidata = []
level =0
i = 1
with open('C:\\Users\\pooja\\Anaconda2\\Lib\\site-packages\\wikipedia\\fintree','rb') as f:
    for line in f:
        try:
            stridx = line.find(':Category:') + len(':Category:')
            qry = line[stridx:(line.find('|'))]
           
      
           
           # print 'query:',qry
            article = wikipedia.page(qry)
            if line.find('###') != -1:
                level = 3
            elif line.find('##') != -1:
                level = 2
            elif line.find('#') != -1:
                level = 1
            else:
                level = 0
            wikidata.append([i,article.title, article.content, level])
        #    print 'title:',article.title
            i = i +1
        except:
            i = i + 1
            pass

wikidf = pd.DataFrame(wikidata,columns=['ID','Title','Text', 'Level'])

wikidf['Text'] = wikidf['Text'].str.lower()
wikidf['Title'] = wikidf['Title'].str.lower()
wikidf["new_column"] = wikidf['Text'].str.replace('[^\w\s]',' ') # remove punct
wikidf['new_column'] = wikidf['new_column'].str.replace('\d+', '') #remove numbers

for index,i in wikidf.iterrows():
    txt = i['new_column']
    wikidf['new_column'].iloc[index] = remove_stp_words(txt)



for index,i in wikidf.iterrows():
    txt = i['new_column']
    wikidf['new_column'].iloc[index] = stem_words(txt)

tok_corp= [nltk.word_tokenize(sent.decode('utf-8')) for sent in wikidf['new_column']]

#model = gensim.models.Word2Vec(tok_corp, min_count=1, size = 32)
gen_docs = [[w.lower() for w in word_tokenize(text)] for text in wikidf['new_column']]
dictionary = gensim.corpora.Dictionary(gen_docs)
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
tf_idf = gensim.models.TfidfModel(corpus)
sims = gensim.similarities.Similarity('C:\\Users\\pooja\\',tf_idf[corpus],
                                      num_features=len(dictionary))




print("Number of words in dictionary:",len(dictionary))

query = raw_input("Enter article title..")
test = wikipedia.page(query)
query_doc = [w.lower() for w in word_tokenize(test.content)]

query_doc_bow = dictionary.doc2bow(query_doc)

query_doc_tf_idf = tf_idf[query_doc_bow]
print("tf/idf complete...")
arr = sims[query_doc_tf_idf].tolist()
maxidx = arr.index(max(arr))
parindex = wikidf['Level'].iloc[maxidx] - 1
parent = wikidf['Title'].iloc[maxidx]


hlist = [test.title,parent]
while(parindex>=0):
    prevTitle = parent
   
    newdatalist = []
    for index,i in wikidf.iterrows():
        if i['Level'] == parindex:
            newdatalist.append([i['ID'],i['Title'],i['Text'],i['Level'],i['new_column']])
    newdf = pd.DataFrame(newdatalist,columns=['ID','Title','Text', 'Level','new_column'])
    gen_docs = [[w.lower() for w in word_tokenize(text)]  for text in newdf['new_column']] 
    dictionary1 = gensim.corpora.Dictionary(gen_docs)
    print("Number of words in dictionary:",len(dictionary1))
    corpus1 = [dictionary1.doc2bow(gen_doc) for gen_doc in gen_docs]
    tf_idf1 = gensim.models.TfidfModel(corpus1)
    sims1 = gensim.similarities.Similarity('C:\\Users\\pooja\\',tf_idf1[corpus1], num_features=len(dictionary1))
    query_doc = [w.lower() for w in word_tokenize(parent)]
    query_doc_bow = dictionary1.doc2bow(query_doc)
    query_doc_tf_idf1 = tf_idf1[query_doc_bow]
    testarr = sims1[query_doc_tf_idf1].tolist()
    maxidx = testarr.index(max(testarr))
    print (newdf['Title'].iloc[maxidx] + " is most similar to "+ parent)
    hlist.append(newdf['Title'].iloc[maxidx])
   # print("index of parent:"+newdf['Level'].iloc[maxidx])
    parindex = newdf['Level'].iloc[maxidx] - 1
print "Hierarchy..."
for i in reversed(hlist):
    print i +'-->'


C:\Users\pooja\Anaconda2\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
No handlers could be found for logger "gensim.models.doc2vec"
C:\Users\pooja\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\pooja\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
C:\Users\pooja\Anaconda2\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is tryin

('Number of words in dictionary:', 21060)
Enter article title..hedge fund
tf/idf complete...
('Number of words in dictionary:', 8541)
financial services is most similar to investment fund
('Number of words in dictionary:', 724)
finance is most similar to investment fund
Hierarchy...
finance-->
financial services-->
investment fund-->
Hedge fund-->


In [1]:
def remove_stp_words(text):
    
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(text)

    filtered_sentence = [w for w in word_tokens if not w in stop_words]

    filtered_sentence = []

# Remove stop words

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    
    return(' '.join(filtered_sentence))

In [2]:
# stemming
def stem_words(text):
    
    ps = PorterStemmer()
    word_tokens = word_tokenize(txt)
    res= ''
    for w in word_tokens:
        res = res + ' '+ ps.stem(w)
    
    return res

In [25]:

query = raw_input("Enter article title..")
test = wikipedia.page(query)
query_doc = [w.lower() for w in word_tokenize(test.content)]

query_doc_bow = dictionary.doc2bow(query_doc)

query_doc_tf_idf = tf_idf[query_doc_bow]
print("tf/idf complete...")
arr = sims[query_doc_tf_idf].tolist()
maxidx = arr.index(max(arr))
parindex = wikidf['Level'].iloc[maxidx] - 1
parent = wikidf['Title'].iloc[maxidx]


hlist = [test.title,parent]
while(parindex>=0):
    prevTitle = parent
   
    newdatalist = []
    for index,i in wikidf.iterrows():
        if i['Level'] == parindex:
            newdatalist.append([i['ID'],i['Title'],i['Text'],i['Level'],i['new_column']])
    newdf = pd.DataFrame(newdatalist,columns=['ID','Title','Text', 'Level','new_column'])
    gen_docs = [[w.lower() for w in word_tokenize(text)]  for text in newdf['new_column']] 
    dictionary1 = gensim.corpora.Dictionary(gen_docs)
    print("Number of words in dictionary:",len(dictionary1))
    corpus1 = [dictionary1.doc2bow(gen_doc) for gen_doc in gen_docs]
    tf_idf1 = gensim.models.TfidfModel(corpus1)
    sims1 = gensim.similarities.Similarity('C:\\Users\\pooja\\',tf_idf1[corpus1], num_features=len(dictionary1))
    query_doc = [w.lower() for w in word_tokenize(parent)]
    query_doc_bow = dictionary1.doc2bow(query_doc)
    query_doc_tf_idf1 = tf_idf1[query_doc_bow]
    testarr = sims1[query_doc_tf_idf1].tolist()
    maxidx = testarr.index(max(testarr))
    print (newdf['Title'].iloc[maxidx] + " is most similar to "+ parent)
    hlist.append(newdf['Title'].iloc[maxidx])
   # print("index of parent:"+newdf['Level'].iloc[maxidx])
    parindex = newdf['Level'].iloc[maxidx] - 1
print "Hierarchy..."
for i in reversed(hlist):
    print i +'-->'

Enter article title..donal trump
tf/idf complete...
('Number of words in dictionary:', 8541)
debt is most similar to rich dad poor dad
('Number of words in dictionary:', 724)
finance is most similar to rich dad poor dad
Hierarchy...
finance-->
debt-->
rich dad poor dad-->
Donald Trump-->


In [8]:
import wikipedia
import pandas as pd
from time import sleep
import time
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim


In [10]:
wikidf

,ID,Title,Text,Level,new_column
0,1,finance,finance is a field that deals with the study o...,0,finance is a field that deals with the study o...
1,2,finance,finance is a field that deals with the study o...,1,finance is a field that deals with the study o...
2,3,namibia,"namibia ( ( listen), ), officially the republi...",2,namibia listen officially the republi...
3,5,ministry of finance (argentina),ministry of economy and public finances was th...,2,ministry of economy and public finances was th...
4,7,greeks (finance),"in mathematical finance, the greeks are the qu...",2,in mathematical finance the greeks are the qu...
5,9,chinese financial system,china's financial system has recently begun to...,2,china s financial system has recently begun to...
6,11,secretary of finance (philippines),the secretary of finance (filipino: kalihim ng...,2,the secretary of finance filipino kalihim ng...
7,13,"ministry of finance, planning and economic dev...","the ministry of finance, planning and economic...",2,the ministry of finance planning and economic...
8,15,cbv vietnam finance indices,cbv vietnam finance indices comprise the only ...,2,cbv vietnam finance indices comprise the only ...
9,17,jordan,jordan (; arabic: الأردن‎ al-‘urdunn [al.ʔur.d...,2,jordan arabic al urdunn al ur d...
